# Find Object-s in cone
Useful links for plotting :
* https://coolum001.github.io/geopandasdisplay.html

In [14]:
import SciServer.CasJobs as cj
from datetime import datetime
import pandas
import matplotlib.pyplot as plt
import random 

In [2]:
LSST="LSST_aris"
# code for perfect(unit?) sphere
SRID=104001

## Find Object rows 
Various different version of table used

In [19]:
#
# use temptable #a iso defining it inside CTE, as otherwise it will be executed multiple times in the union
#
def conesearch(object_table,ra,decl,sr):
    """
    sr in arcmin
    """
    print("[",datetime.now(),"] table:",object_table,", conesearch ({ra},{decl},{sr}:".format(ra=ra,decl=decl,sr=sr))
    sql="""
select o.deepsourceid as objId, o.ra, o.decl 
  from htm.CoverCircleEq({ra},{decl},{sr}) c
  join {table} o on o.htmid between c.htmidstart and htmidend
    """.format(table=object_table,ra=ra,decl=decl,sr=sr)
    return cj.executeQuery(sql,LSST)

def conesearch_job(object_table,ra,decl,sr):
    """
    sr in arcmin
    """
    newtable="{table}_{rnd}".format(table=object_table,rnd=random.randint(0,10000000))
    print("[",datetime.now(),"] table:",object_table,", conesearch ({ra},{decl},{sr}:".format(ra=ra,decl=decl,sr=sr),
         "output to:",newtable)
    
    sql="""
select o.deepsourceid as objId, o.ra, o.decl 
into MyDB.{newtable}
  from htm.CoverCircleEq({ra},{decl},{sr}) c
  join {table} o on o.htmid between c.htmidstart and htmidend
    """.format(table=object_table,ra=ra,decl=decl,sr=sr,newtable=newtable)
    return cj.submitJob(sql,LSST),newtable

In [ ]:
RA=45;DECL=45;SR=3

m_Object_cci, plan
<img src="./img/cone_cci_plan.png"></img>

In [ ]:
%time _cci=conesearch('m_object_cci',RA,DECL,SR)

m_Object_cli, plan
<img src="./img/cone_cli_plan.png"></img>

In [ ]:
%time _cli=conesearch('m_object_cli',RA,DECL,SR)

m_Object_page, plan
<img src="./img/cone_page_plan.png"></img>

In [12]:
%time _page=conesearch('m_object_page',RA,DECL,SR)

[ 2020-01-19 15:31:51.794901 ] table: m_object_cci , conesearch (45,45,3:
CPU times: user 32.3 ms, sys: 12.4 ms, total: 44.7 ms
Wall time: 2.35 s
[ 2020-01-19 15:31:54.141636 ] table: m_object_cli , conesearch (45,45,3:
CPU times: user 33.8 ms, sys: 3.72 ms, total: 37.5 ms
Wall time: 2.25 s
[ 2020-01-19 15:31:56.396506 ] table: m_object_page , conesearch (45,45,3:
CPU times: user 33.4 ms, sys: 1.39 ms, total: 34.7 ms
Wall time: 2.28 s


ObjectDP_htmid, plan
<img src="./img/cone_dp_plan.png"></img>

In [22]:
jobid,newtable=_page=conesearch_job('objectdp_htmid',RA,DECL,SR)
%time r=cj.waitForJob(jobid,verbose=True)

[ 2020-01-19 15:39:28.992672 ] table: objectdp_htmid , conesearch (45,45,3: output to: objectdp_htmid_8347010
Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...

{'JobID': 46421257,
 'Rows': None,
 'WebServicesID': 999687452,
 'TimeSubmit': '2020-01-19T10:39:30.857',
 'TimeStart': '2020-01-19T10:39:53.283',
 'TimeEnd': '2020-01-19T10:51:00.42',
 'SendEmail': 0,
 'Status': 4,
 'AutoComplete': 0,
 'Estimate': 500,
 'TaskName': 'Compute.SciScript-Python.CasJobs.submitJob',
 'OutputLoc': '',
 'HostIP': 'LSST_Aris_long ',
 'Message': 'Cancellation complete.',
 'Query': '\nselect o.deepsourceid as objId, o.ra, o.decl \ninto MyDB.objectdp_htmid_8347010\n  from htm.CoverCircleEq(45,45,3) c\n  join objectdp_htmid o on o.htmid between c.htmidstart and htmidend\n    ',
 'ModifiedQuery': 'select o.deepsourceid as objId, o.ra, o.decl \n\n  from htm.CoverCircleEq(45,45,3) c\n  join objectdp_htmid o on o.htmid between c.htmidstart and htmidend/*CASJOBS_INTO_TABLE:[mydbsql].mydb_999687452.webuser.objectdp_htmid_8347010*/',
 'Target': 'LSST_aris',
 'OutputType': 'QUERY',
 'Params': '{\r\n  "token": "a08968cd1bf94e6f95d0834d93177705"\r\n}',
 'Created_Table': 'ob

In [ ]:
c

# Plotting